## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [22]:
import numpy as np

array = np.random.randint(0, 1000, size=1_000_000)

In [23]:
def mean_0(arr):
    acc, cnt = 0, 0
    for ai in arr:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt

In [24]:
%timeit mean_0(array)

254 ms ± 38.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
def mean_1(arr):
    acc = 0
    for ai in arr:
        bi = ai + 100
        acc += bi
    return acc / len(arr)

In [26]:
%timeit mean_1(array)

245 ms ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [28]:
%lprun -f mean_1 mean_1(array)

In [29]:
def mean_2(arr):
    return sum(arr) / len(arr) + 100

In [30]:
%timeit mean_2(array)

69.4 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
def mean_3(arr):
    return np.mean(arr) + 100

In [42]:
%timeit mean_3(array)

861 µs ± 84.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
import numba


@numba.njit
def mean_0_numba(arr):
    acc, cnt = 0, 0
    for ai in arr:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc / cnt


%timeit mean_0_numba(array)

471 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [57]:
import pandas as pd
from string import ascii_lowercase

N = 2_000_000
df = pd.DataFrame(
    data=np.random.random(size=(N, 4)),
    columns=[f'random_{i}' for i in range(4)]
)
df['key'] = np.random.default_rng().choice(list(ascii_lowercase), size=N, replace=True)

In [76]:
def solution_0(df_):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df_[df_['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [77]:
%timeit solution_0(df)

574 ms ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
def solution_1(df_):
    first_letters = ascii_lowercase[:5]
    return df_[df_['key'].isin(set(first_letters))]

In [83]:
%timeit solution_1(df)

85.7 ms ± 869 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [125]:
# @numba.njit
def foo(n_array):
    first_letters = {ord(x) for x in ascii_lowercase[:5]}
    print(first_letters)
    print(n_array)
    return n_array == first_letters


def solution_2(df_):
    # print(foo(df_['key'].to_numpy()))
    return foo(df_['key'].to_numpy())


print(foo(df['key'].apply(lambda x: ord(x)).to_numpy()))

{97, 98, 99, 100, 101}
[101 100 114 ...  99 103 105]
[False False False ... False False False]


In [124]:
%timeit solution_2(df)

{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g' 'i']
{97, 98, 99, 100, 101}
['e' 'd' 'r' ... 'c' 'g

## Лабораторная работа 3

In [35]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [36]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва. 
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba`
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
